In [1]:
import mask_dataset
from config import config
from model import MaskCNN

import os
import random
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, SubsetRandomSampler

In [2]:
random.seed(config.random_seed)
torch.manual_seed(config.random_seed)

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
dataset = mask_dataset.MaskDataset(root_dir='../dataset', img_dim=config.img_dim)

In [5]:
foldperf = {}
splits = KFold(n_splits=10, shuffle=True, random_state=0)

In [6]:
avg_loss = 0
best_vloss = 999999
y_total_pred = []
y_total_true = []


for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
    print('Fold {}'.format(fold + 1))
    model = MaskCNN(len(dataset.labels), config.img_dim, base_filter_size=config.base_filter_size)
    model = model.to(device)

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size, sampler=torch.utils.data.SubsetRandomSampler(train_idx))
    valid_loader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size, sampler=torch.utils.data.SubsetRandomSampler(val_idx))  

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

    for epoch in range(config.epochs):
        # traning
        model.train()
        correct = 0
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()
            preds = model(images)
            loss = loss_fn(preds, labels)
            loss.backward()
            optimizer.step()
            correct += (torch.max(preds, dim=1)[1] == labels).sum().item()

            running_loss += loss.item()
            preds = torch.argmax(preds.data, dim=1)
            y_total_true.extend(labels.data.cpu().numpy())
            y_total_pred.extend(preds.data.cpu().numpy())

        # validation
        model.eval()
        val_correct = 0
        running_vloss = 0
        with torch.no_grad():
            for i, vdata in enumerate(valid_loader):
                vimages, vlabels = vdata[0].to(device), vdata[1].to(device)
                vpreds = model(vimages)
                vloss = loss_fn(vpreds, vlabels)
                running_vloss += vloss.item()
                val_correct += (torch.max(vpreds, dim=1)[1] == vlabels).sum().item()
                vpreds = torch.argmax(vpreds.data, dim=1)
                y_total_true.extend(vlabels.data.cpu().numpy())
                y_total_pred.extend(vpreds.data.cpu().numpy())
        running_loss = running_loss / len(train_loader.sampler)
        train_acc = correct / len(train_loader.sampler) * 100
        running_vloss = running_vloss / len(valid_loader.sampler)
        val_acc = val_correct / len(valid_loader.sampler) * 100
        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                                                 config.epochs,
                                                                                                                                 running_loss,
                                                                                                                                 running_vloss,
                                                                                                                                 train_acc,
                                                                                                                                 val_acc))
        history['train_loss'].append(running_loss)
        history['test_loss'].append(running_vloss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(val_acc)

    foldperf['fold{}'.format(fold+1)] = history  


torch.save(model,'MaskCNN_kfold_old.pt')
print('\ndone.')

Fold 1
Epoch:1/3 AVG Training Loss:0.015 AVG Test Loss:0.016 AVG Training Acc 60.19 % AVG Test Acc 58.67 %
Epoch:2/3 AVG Training Loss:0.008 AVG Test Loss:0.014 AVG Training Acc 79.93 % AVG Test Acc 71.00 %
Epoch:3/3 AVG Training Loss:0.004 AVG Test Loss:0.012 AVG Training Acc 91.93 % AVG Test Acc 73.67 %
Fold 2
Epoch:1/3 AVG Training Loss:0.016 AVG Test Loss:0.017 AVG Training Acc 58.56 % AVG Test Acc 55.00 %
Epoch:2/3 AVG Training Loss:0.009 AVG Test Loss:0.013 AVG Training Acc 78.52 % AVG Test Acc 69.33 %
Epoch:3/3 AVG Training Loss:0.004 AVG Test Loss:0.011 AVG Training Acc 92.37 % AVG Test Acc 74.33 %
Fold 3
Epoch:1/3 AVG Training Loss:0.015 AVG Test Loss:0.018 AVG Training Acc 61.30 % AVG Test Acc 53.00 %
Epoch:2/3 AVG Training Loss:0.008 AVG Test Loss:0.014 AVG Training Acc 81.26 % AVG Test Acc 66.00 %
Epoch:3/3 AVG Training Loss:0.004 AVG Test Loss:0.012 AVG Training Acc 91.15 % AVG Test Acc 72.67 %
Fold 4
Epoch:1/3 AVG Training Loss:0.015 AVG Test Loss:0.018 AVG Training Acc 6

In [7]:
testl_f,tl_f,testa_f,ta_f=[],[],[],[]
k = 10
for f in range(1,k+1):

     tl_f.append(np.mean(foldperf['fold{}'.format(f)]['train_loss']))
     testl_f.append(np.mean(foldperf['fold{}'.format(f)]['test_loss']))

     ta_f.append(np.mean(foldperf['fold{}'.format(f)]['train_acc']))
     testa_f.append(np.mean(foldperf['fold{}'.format(f)]['test_acc']))

print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.3f} \t Average Test Loss: {:.3f} \t Average Training Acc: {:.2f} \t Average Test Acc: {:.2f}".format(np.mean(tl_f),np.mean(testl_f),np.mean(ta_f),np.mean(testa_f)))     

Performance of 10 fold cross validation
Average Training Loss: 0.009 	 Average Test Loss: 0.014 	 Average Training Acc: 77.59 	 Average Test Acc: 65.24


In [8]:
import mask_dataset
from config import config
from model import MaskCNN

import os
import random
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

import torch
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [9]:
print(f'accuracy: {accuracy_score(y_total_true, y_total_pred):.3f}')
print(f'recall: {recall_score(y_total_true, y_total_pred, average="weighted"):.3f}')
print(f'precision: {precision_score(y_total_true, y_total_pred, average="weighted"):.3f}')
print(f'f1: {f1_score(y_total_true, y_total_pred, average="weighted"):.3f}')

accuracy: 0.764
recall: 0.764
precision: 0.766
f1: 0.764
